In [4]:
# You will need to install iqm-benchmarks from the github repo to access all the mGST functions: https://github.com/iqm-finland/iqm-benchmarks
from mGST.low_level_jit import dK, objf
from mGST.optimization import tangent_proj
import shelve
import numpy as np

In [19]:
# Loading parameters
filename = "shelve_out"
my_shelf = shelve.open(filename)
for key in ["K", "E_new", "rho", "y", "J", "d", "r", "rK"]:
    print(key)
    globals()[key]=my_shelf[key]
my_shelf.close()

K
E_new
rho
y
J
d
r
rK


In [6]:
# Setting some additional_parameters
E = E_new
pdim = int(np.sqrt(r))
n = rK * pdim
Delta = np.zeros((d, n, pdim)).astype(np.complex128)
X = np.einsum("ijkl,ijnm -> iknlm", K, K.conj()).reshape((d, r, r))

The function dK computes the Wirtinger derivative $\frac{\partial \mathcal L}{\partial K}$.
Here $\mathcal L$ is the cost function "objf".

In [7]:
dK_ = dK(X, K, E, rho, J, y, d, r, rK)

In [8]:
print(objf(X, E, rho, J, y))

0.0019762572998321807


In [9]:
# This loop computes the Riemannian gradient from the derivative
for k in range(d):
    # derivative
    Fy = dK_[k].reshape(n, pdim)
    Y = K[k].reshape(n, pdim)
    # Riem. gradient taken from conjugate derivative
    rGrad = Fy.conj() - Y @ Fy.T @ Y
    Delta[k] = rGrad

In [10]:
# There is also a function to project onto the tangent space should you need it:
Delta_proj = tangent_proj(K, Delta, d, rK)

In [11]:
# As expected, the Riemannian gradient is already in the tangent space :)
print(np.linalg.norm(Delta - Delta_proj))

3.170989642158836e-17
